# Address missing data

In [8]:
import urllib
import urllib.request
import urllib.parse
import urllib.error
import requests
from bs4 import BeautifulSoup
import csv
import os
import pandas as pd
import clean_documents
from pathlib import Path
import spacy
from spacy import displacy
from spacy.matcher import Matcher
from functools import reduce
import extract_dates
from start import data_path

In [9]:
exemptions_df = pd.read_csv(os.path.join(data_path, 'doi_exemptions_scraped.csv'))
dates_df = pd.read_csv(os.path.join(data_path, 'doi_dates_scraped.csv'))

In [10]:
docs_df = exemptions_df.merge(dates_df)
docs_df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,title,level,type,link,text,p_innovation,...,term_phrase,term_p,finalize_year,finalize_month,finalize_phrase,finalize_p,date_year,date_month,date_phrase,date_ay
0,0,1323,1323,1451,Lake Travis ISD,Second,pdf,https://www.ltisdschools.org//cms/lib/Tx018000...,LTISD Plan of Innovation LAKE TRAVIS INDEPENDE...,0.999890,...,: 1. Beginning with the 2017-2018 academic yea...,0.999955,2016,December,"hold a public meeting on December 13, 2016 to ...",0.996723,2017.0,"(999, '')",: 1. Beginning with the 2017-2018 academic yea...,2017.0
1,1,1913,1913,2108,Zephyr ISD,Second,docx,http://zephyrisd.net/wp-content/uploads/2014/0...,Zephyr ISD District of Innovation Plan Introdu...,0.999922,...,Term The District of Innovation Plan will beco...,0.999955,2018,April,"go to the Board on April 16th, 2018. Term The ...",0.889274,2018.0,"(2018, 'September')",Term The District of Innovation Plan will beco...,2018.0
2,2,1895,1895,2089,Zavalla ISD,Second,pdf,https://s3.amazonaws.com/scschoolfiles/1772/za...,Zavalla ISD District of Innovation Plan (HB 18...,0.999619,...,This plan will be in effect for the 2017-2018 ...,0.999955,2017,February,"27,2017 Final version plan posted January 27, ...",0.999955,2017.0,"(999, '')",This plan will be in effect for the 2017-2018 ...,2017.0
3,3,577,577,596,Zapata County ISD,Second,pdf,https://1.cdn.edl.io/VZ6fi1M6pn8Qk8tS3GetGewCD...,Microsoft Word - Document1 ZAPATA COUNTY INDEP...,0.452118,...,"a District of Innovation on Tuesday, December ...",0.000187,2017,December,"a District of Innovation on Tuesday, December ...",0.999884,2017.0,December,"a District of Innovation on Tuesday, December ...",2017.0
4,4,1784,1784,1967,Yorktown ISD,Second,pdf,http://www.yisd.org/userfiles/57/my%20files/fi...,Yorktown Independent School District Final Dis...,0.999877,...,Yorktown Independent School District Final Dis...,0.999955,2017,May,"of the letter. On May 15, 2017 the District Co...",0.972258,2017.0,"(999, '')",Yorktown Independent School District Final Dis...,2017.0
5,5,163,163,163,Yoakum ISD,First,pdf,http://www.yoakumisd.net/cms/lib3/TX01001553/C...,UNAVAILABLE,0.005025,...,NaN,0.000000,-999,NaN,NaN,0.000000,-999.0,NaN,NaN,-999.0
6,6,2173,2173,2410,Yantis ISD,Second,pdf,http://www.yantisisd.net/users/2017-2018/Distr...,Yantis ISD District of Innovation Plan (HB1842...,0.998937,...,The District of Innovation Plan will become ef...,0.999955,2018,June,"presented to the SBDM team on June, 2018. Yant...",0.958766,2017.0,"(2017, 'June')",The District of Innovation Plan will become ef...,2017.0
7,7,3713,3713,109,Wylie ISD (221912),html,html,http://www.wyliebulldogs.org/cms/One.aspx?port...,District of Innovation - Wylie Independent Sch...,0.001331,...,2019-2020 School Calendar Parents 2018-2019 Di...,0.999831,2019,NaN,Summer Feeding Program Menus Make a Lunch Paym...,0.964533,2018.0,"(999, '')",2019-2020 School Calendar Parents 2018-2019 Di...,2018.0
8,8,32,32,32,Wylie ISD (043914),First,pdf,http://www.wylieisd.net/cms/lib09/TX01918453/C...,Wylie ISD INNOVATION PLAN INTRODUCTION House B...,0.999891,...,INNOVATION PLAN INTRODUCTION House Bill (HB) 1...,0.134219,2017,March,"Plan at its meeting on March 20, 2017. DISTRIC...",0.999955,2017.0,March,"Plan at its meeting on March 20, 2017. DISTRIC...",2016.0
9,9,1659,1659,1828,Wortham ISD,Second,docx,https://s3.amazonaws.com/scschoolfiles/888/wis...,W Wortham ISD District of Innovation Plan 2017...,0.997949,...,W Wortham ISD District of Innovation Plan 2017...,0.999955,-999,NaN,NaN,0.000000,2017.0,"(999, '')",W Wortham ISD District of Innovation Plan 2017...,2017.0


## Check TEA website for new districts of innovation

In [11]:
add_new_dois = False

In [12]:
if add_new_dois:
    url = "https://tea.texas.gov/Texas_Schools/District_Initiatives/Districts_of_Innovation/"
    webcontent = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(webcontent, 'html.parser')
    links = soup.find_all('a')
    districts_list = [i for i in map(lambda x: x.get('title'), links) 
                      if i is not None and 'ISD' in i]
    districts_df = pd.DataFrame(districts_list, columns=['title'])
    len(districts_df)
    # Make a new DataFrame from a right join on dois with list of districts
    combined_df = docs_df.merge(districts_df, how='right', left_on='title', right_on='title')

    # Save the district name and link for those districts that we don't have laws for
    missing_districts = combined_df[(combined_df['possible_laws'] == '[]')][['title', 'link']]

    print("Number of missing districts: ", len(missing_districts))
    missing_districts.head()

In [13]:
if add_new_dois == False: 
    # Save the district name and link for those districts that we don't have laws for
    missing_districts = docs_df[(docs_df['possible_laws'] == '[]')][['title', 'link']]
    print("Number of missing districts: ", len(missing_districts))
    missing_districts.head()

Number of missing districts:  277


In [14]:
missing_districts.head()

,title,link
5,Yoakum ISD,http://www.yoakumisd.net/cms/lib3/TX01001553/C...
7,Wylie ISD (221912),http://www.wyliebulldogs.org/cms/One.aspx?port...
10,Woodville ISD,https://core-docs.s3.amazonaws.com/documents/a...
11,Woodson ISD,https://sites.google.com/a/woodsonisd.net/wood...
14,Woden ISD,http://www.wodenisd.org/home/district-announce...


## Address districts that are missing due to incorrect link.

### Where possible, incorporate previously (manually) collected links before editing

In [15]:
missing_districts = missing_districts.rename({'link': 'link_scraped'}, axis = 'columns')
missing_links = pd.read_csv(os.path.join(data_path, 'missing_links_corrected.csv'))
missing_links = missing_links.rename({'link': 'link_manual'}, axis = 'columns')
missing_links = missing_districts.merge(missing_links, how = 'left', left_on = 'title', right_on = 'title')
print("Number of missing links", len(missing_links))
missing_links['link'] = missing_links.link_manual
missing_links['link'] = missing_links['link'].fillna(missing_links['link_scraped'])
missing_links.head()

Number of missing links 277


,title,link_scraped,Unnamed: 0,link_manual,link
0,Yoakum ISD,http://www.yoakumisd.net/cms/lib3/TX01001553/C...,0,http://www.yoakumisd.net/cms/lib3/TX01001553/C...,http://www.yoakumisd.net/cms/lib3/TX01001553/C...
1,Wylie ISD (221912),http://www.wyliebulldogs.org/cms/One.aspx?port...,1,http://www.wyliebulldogs.org/UserFiles/Servers...,http://www.wyliebulldogs.org/UserFiles/Servers...
2,Woodville ISD,https://core-docs.s3.amazonaws.com/documents/a...,2,https://core-docs.s3.amazonaws.com/documents/a...,https://core-docs.s3.amazonaws.com/documents/a...
3,Woodson ISD,https://sites.google.com/a/woodsonisd.net/wood...,3,https://drive.google.com/file/d/0B5Y_iCwf_UMuT...,https://drive.google.com/file/d/0B5Y_iCwf_UMuT...
4,Woden ISD,http://www.wodenisd.org/home/district-announce...,4,https://drive.google.com/file/d/1j-lifioD9y4Z7...,https://drive.google.com/file/d/1j-lifioD9y4Z7...


In [16]:
missing_links = missing_links[['title', 'link']]
missing_links.to_csv(os.path.join(data_path, 'missing_links.csv'))

### Manually edit links in missing_links.csv where wrong. Save as missing_links_corrected.csv. Then upload.

In [17]:
missing_links = pd.read_csv(os.path.join(data_path, 'missing_links_corrected.csv'))
missing_links.tail()

,Unnamed: 0,title,link
272,272,Anahuac ISD,https://drive.google.com/file/d/11uTSTuAcBFTdq...
273,273,Amarillo ISD,http://www.amaisd.org/UserFiles/Servers/Server...
274,274,Alvarado ISD,http://www.alvaradoisd.net/UserFiles/Servers/S...
275,275,Alto ISD,https://resources.finalsite.net/images/v154092...
276,276,Aledo ISD,https://docs.google.com/viewerng/viewer?url=ht...


### Extract text from missing_links

In [18]:
texts = []
for link in missing_links.link:
    text = clean_documents.get_plain_text(link)
    texts.append(text)

Google Error: 'd' is not in listle.com/drive/folders/0B-ALJgmFKVDFZmxBb09rRE9aQ1Uiewategories/Documents/District_of_Innovation_Timeline_&_Plan16_PDF.pdfll%201842.pdfExemptions-BoardApproved.pdf
Google Error: 'd' is not in listle.com/open?id=1kC4Rn_OgoK8K-s3kvUh4xBPTluPJdtpKtes/News/Documents/FINAL_DOI_PDF_FIle.pdf20Final.pdf7-2022_Adopted%2005.09.2017_Revised%2006.2018.docx
Google Error: 'd' is not in listle.com/drive/folders/169mMXgf5g3TeRrsjYPg7fpq25FI57Ce_?usp=sharing_2017.pdfan%202018-2023.pdfpdfMillsap+ISD+Innovation+Plan+2018.docxru%202021.pdfproved.pdf2F2018%20Website%20%2D%20Required%2FDistrict%2Dof%2DInnovation%2DPlan%2D2016%2D2017%2DBoard%2DApproved%2Epdf&parent=%2Fpersonal%2Fmarcus%5Fdavis%5Fpottsboroisd%5Forg%2FDocuments%2F2018%20Website%20%2D%20Required&slrid=58be829e%2D30c5%2D6000%2D8997%2De55bfde53779
error:  link: https://www.galenaparkisd.com/cms/lib/TX02217083/Centricity/Domain/799/_District%20of%20Innovation%20Plan%20-%20Current%20w%20Amend%202.pdfIBPXGFLSZTYQ&Expire

2019-03-29 19:34:58,665 [MainThread  ] [WARNI]  Tika server returned status: 422


error: HTTP Error 404: File Not Found link: http://www.ezzellisd.org/apps/pages/index.jsp?uREC_ID=199567&type=d&pREC_ID=431083
Google Error: 'd' is not in listle.com/drive/folders/0B3zKmpprcZY1OHRBcEw1d2VFcFEISD/district%20of%20innovation/CLEAR%20CREEK%20INDEPENDENT%20SCHOOL%20DISTRICT%20PLAN%20FINAL.pdf


In [19]:
missing_links['text'] = texts
missing_links['text'] = missing_links['text'].fillna('No text')
len(missing_links)

277

In [20]:
missing_links = clean_documents.remove_whitespace(missing_links, text_col = 'text')
missing_links.head()

,Unnamed: 0,title,link,text
0,0,Yoakum ISD,http://www.yoakumisd.net/cms/lib3/TX01001553/C...,DISTRICT OF INNOVATION DISTRICT OF INNOVATION ...
1,1,Wylie ISD (221912),http://www.wyliebulldogs.org/UserFiles/Servers...,District of Innovation Introduction HB 1842 wa...
2,2,Woodville ISD,https://core-docs.s3.amazonaws.com/documents/a...,No text
3,3,Woodson ISD,https://drive.google.com/file/d/0B5Y_iCwf_UMuT...,Woodson Independent School District District o...
4,4,Woden ISD,https://drive.google.com/file/d/1j-lifioD9y4Z7...,DISTRICT OF INNOVATION PLAN Introduction The 8...


### Extract laws from missing_links

In [21]:
nlp = spacy.load('en_core_web_sm')
law_shapes = [i*'d' + '.' + j*'d' for i in range(1, 4) for j in range(3,5)]
law_shape_patterns = [[{'LIKE_NUM':True, 'SHAPE':shape}, {'ORTH':'%', 'OP':'!'}] for shape in law_shapes] # could add {'SHAPE':'§', 'OP':'*'},  and exceptions for laws in parentheses or followed by a hyphen

matcher = Matcher(nlp.vocab)
matcher.add("ExplicitLaw", None, *law_shape_patterns)

def get_matches(string):
    doc = nlp(string)
    matches = matcher(doc)
    return list(set([doc[i[1]:i[2]][0] for i in matches]))

missing_links['possible_laws'] = missing_links.text.apply(get_matches)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


### Extract dates from manually collected links

In [22]:
output_dir = os.path.join(data_path, 'date_term_classifier')
start_dates = []
date_phrases = []
p_terms = []
for text in missing_links.text:
    date, month, phrase, p = extract_dates.get_term_date_and_phrase(text, output_dir)
    start_dates.append(date)
    date_phrases.append(phrase)        
    p_terms.append(p)
missing_links['date'] = start_dates
missing_links['date_phrase'] = date_phrases
missing_links['date_p'] = p_terms

In [23]:
len(missing_links)

277

In [24]:
print('There are', len(missing_links[missing_links.text == 'No text']), 'documents without text (photocopies, most likely)')
print('There are', len(missing_links[missing_links.text == 'UNAVAILABLE']), 'documents which we cannot access')
print('An additional', len(missing_links[missing_links['possible_laws'].astype(str) == '[]'][(missing_links.text != 'No text') & (missing_links.text != 'UNAVAILABLE')]), 'documents have text but do not contain laws.')
print('and', len(missing_links[missing_links['date'] == -999][missing_links.text != 'No text']), 'documents have text but do not contain dates.')

There are 60 documents without text (photocopies, most likely)
There are 2 documents which we cannot access
An additional 57 documents have text but do not contain laws.
and 53 documents have text but do not contain dates.


/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


### Save now correct observations to 'missing_links_corrected'. Save districts that are still incorrect to missing_text.

In [25]:
missing_links_corrected = missing_links[missing_links['possible_laws'].astype(str) != '[]']
missing_text = missing_links[missing_links['possible_laws'].astype(str) == '[]']

In [26]:
len(missing_text)

119

In [27]:
print('We have added', len(missing_links_corrected), 'districts.')
print(len(missing_text), 'are still missing.')

We have added 158 districts.
119 are still missing.


## Address districts that are missing due to missing or incorrect text.

### Where possible, incorporate previously (manually) collected text before editing
#### TODO: add manual fixed column

In [30]:
missing_text_new = missing_text.rename({'text': 'text_scraped'}, axis = 'columns')
missing_text_old = pd.read_csv(os.path.join(data_path, 'missing_text_corrected.csv'), encoding='latin-1')
missing_text_old = missing_links.rename({'text': 'text_manual'}, axis = 'columns')
missing_text = missing_text_new.merge(missing_text_old, how = 'left', left_on = 'title', right_on = 'title')
print("Number of missing texts", len(missing_text))
missing_text['text'] = missing_text.text_manual
missing_text['text'] = missing_text['text'].fillna(missing_text['text_scraped'])
missing_text.to_csv(os.path.join(data_path, 'missing_text.csv'))

Number of missing texts 119


,Unnamed: 0_x,title,link_x,text_scraped,possible_laws_x,date_x,date_phrase_x,date_p_x,Unnamed: 0_y,link_y,text_manual,possible_laws_y,date_y,date_phrase_y,date_p_y,text
0,2,Woodville ISD,https://core-docs.s3.amazonaws.com/documents/a...,No text,[],-999,,0.000000,2,https://core-docs.s3.amazonaws.com/documents/a...,No text,[],-999,,0.000000,No text
1,5,Wink-Loving ISD,https://1.cdn.edl.io/vQwTZvpBxb8PX0K2KFrGbN9Y0...,Wink-Loving ISD WLISD - The Best in Texas: Bui...,[],-999,the 84 th Legislative Session in House Bill 18...,0.002479,5,https://1.cdn.edl.io/vQwTZvpBxb8PX0K2KFrGbN9Y0...,Wink-Loving ISD WLISD - The Best in Texas: Bui...,[],-999,the 84 th Legislative Session in House Bill 18...,0.002479,Wink-Loving ISD WLISD - The Best in Texas: Bui...
2,6,Windthorst ISD,http://www.windthorstisd.net/Uploads/50/misc/f...,No text,[],-999,,0.000000,6,http://www.windthorstisd.net/Uploads/50/misc/f...,No text,[],-999,,0.000000,No text
3,8,Willis ISD,https://tx50000123.schoolwires.net//cms/lib/TX...,"2 Approved by WISD School Board April 12, 2017...",[],2017,"Approved by WISD School Board April 12, 2017 ...",0.000045,8,https://tx50000123.schoolwires.net//cms/lib/TX...,"2 Approved by WISD School Board April 12, 2017...",[],2017,"Approved by WISD School Board April 12, 2017 ...",0.000045,"2 Approved by WISD School Board April 12, 2017..."
4,9,Whitney ISD,https://www.whitney.k12.tx.us/cms/lib3/TX01001...,No text,[],-999,,0.000000,9,https://www.whitney.k12.tx.us/cms/lib3/TX01001...,No text,[],-999,,0.000000,No text


##### Manually add correct text. If no text (photocopy), put 'No text' in cell. If you cannot find the document, put 'No document found'. Save as missing_text_corrected.csv.

In [31]:
missing_texts = pd.read_csv(os.path.join(data_path, 'missing_text_corrected.csv'))

### Extract laws

In [32]:
missing_texts['possible_laws'] = missing_texts.text.apply(get_matches)

### Extract dates

In [33]:
output_dir = os.path.join(data_path, 'date_term_classifier')
start_dates = []
date_phrases = []
p_terms = []
for text in missing_texts.text:
    date, month, phrase, p = extract_dates.get_term_date_and_phrase(text, output_dir)
    start_dates.append(date)
    date_phrases.append(phrase)        
    p_terms.append(p)
missing_texts['date'] = start_dates
missing_texts['date_phrase'] = date_phrases
missing_texts['date_p'] = p_terms

In [34]:
missing_texts.head()

,Unnamed: 0,Unnamed: 0.1,title,link,text,possible_laws,date,date_phrase,date_p
0,2,2,Woodville ISD,https://core-docs.s3.amazonaws.com/documents/a...,No text,[],-999,,0.000000
1,5,5,Wink-Loving ISD,https://1.cdn.edl.io/vQwTZvpBxb8PX0K2KFrGbN9Y0...,Wink-Loving ISD WLISD - The Best in Texas: Bui...,[],-999,the 84 th Legislative Session in House Bill 18...,0.002479
2,6,6,Windthorst ISD,http://www.windthorstisd.net/Uploads/50/misc/f...,No text,[],-999,,0.000000
3,8,8,Willis ISD,https://tx50000123.schoolwires.net//cms/lib/TX...,"2 Approved by WISD School Board April 12, 2017...",[],2017,"Approved by WISD School Board April 12, 2017 п...",0.000045
4,9,9,Whitney ISD,https://www.whitney.k12.tx.us/cms/lib3/TX01001...,No text,[],-999,,0.000000


In [70]:
missing_text_corrected = missing_texts[missing_texts['possible_laws'].astype(str) != '[]']
missing_laws = missing_texts[missing_texts['possible_laws'].astype(str) == '[]']

In [71]:
print('We have added', len(missing_text_corrected), 'districts.')
print(len(missing_laws), 'are still missing.')

We have added 28 districts.
89 are still missing.


# Address districts that are photocopies

#### Incorporate previous edits

In [74]:
missing_laws_old = pd.read_csv(os.path.join(data_path, 'missing_laws_and_dates_corrected.csv'), encoding='latin-1')
missing_laws_old = missing_laws_old.rename({'possible_laws': 'laws_old', 'date': 'date_old'}, axis = 'columns')
missing_laws = missing_laws.merge(missing_laws_old[['title', 'date_old', 'laws_old']], how = 'left', left_on = 'title', right_on = 'title')
print("Number of missing laws", len(missing_laws))
missing_laws['possible_laws'] = missing_laws.laws_old
missing_laws['date'] = missing_laws.date_old
missing_laws[['title', 'possible_laws', 'date', 'link']].to_csv(os.path.join(data_path, 'missing_laws_and_dates.csv'))
missing_laws.head()

Number of missing laws 89


,Unnamed: 0,Unnamed: 0.1,title,link,text,possible_laws,date,date_phrase,date_p,date_old,laws_old
0,2,2,Woodville ISD,https://core-docs.s3.amazonaws.com/documents/a...,No text,"[25.0811, 25.0812]",2017.0,,0.000000,2017.0,"[25.0811, 25.0812]"
1,5,5,Wink-Loving ISD,https://1.cdn.edl.io/vQwTZvpBxb8PX0K2KFrGbN9Y0...,Wink-Loving ISD WLISD - The Best in Texas: Bui...,"[25.0811, 21.003]",2017.0,the 84 th Legislative Session in House Bill 18...,0.002479,2017.0,"[25.0811, 21.003]"
2,6,6,Windthorst ISD,http://www.windthorstisd.net/Uploads/50/misc/f...,No text,"[21.401, 25.0811]",2017.0,,0.000000,2017.0,"[21.401, 25.0811]"
3,8,8,Willis ISD,https://tx50000123.schoolwires.net//cms/lib/TX...,"2 Approved by WISD School Board April 12, 2017...","[25.0811, 21.003, 21.102]",2017.0,"Approved by WISD School Board April 12, 2017 п...",0.000045,2017.0,"[25.0811, 21.003, 21.102]"
4,9,9,Whitney ISD,https://www.whitney.k12.tx.us/cms/lib3/TX01001...,No text,[25.0811],2017.0,,0.000000,2017.0,[25.0811]


In [77]:
missing_laws_corrected = pd.read_csv(os.path.join(data_path, 'missing_laws_and_dates_corrected.csv'))
missing_laws_corrected.tail()

,Unnamed: 0,title,possible_laws,date,link
84,84,Beeville ISD,"[21.003, 21.044, 25.0811, 25.0812, 21.203, 21....",2017,https://s3.amazonaws.com/scschoolfiles/380/bis...
85,85,Beckville ISD,"[25.092, 25.082, 25.0811, 25.0812, 25.081, 37....",2017,https://drive.google.com/file/d/1bosBi0QjvukFz...
86,86,Aquilla ISD,"[25.0811, 25.083, 25.081, 25.082, 25.112, 25.1...",2017,https://core-docs.s3.amazonaws.com/documents/a...
87,87,Anthony ISD,"[21.003, 25.0811]",2016,http://www.anthonyisd.net/assets/aisd-district...
88,88,Amarillo ISD,"[25.0811, 25.0812, 25.081, 25.082]",2016,http://www.amaisd.org/UserFiles/Servers/Server...


# Update full dataset

In [78]:
missing_links_corrected = missing_links_corrected.set_index('title')
missing_text_corrected = missing_text_corrected.set_index('title')
missing_laws_corrected = missing_laws_corrected.set_index('title')
docs_df = docs_df.set_index('title')
docs_df = docs_df.sort_index()

In [79]:
docs_df.update(missing_links_corrected)
docs_df.update(missing_text_corrected)
docs_df.update(missing_laws_corrected)

In [80]:
print("Only missing", len(docs_df[docs_df['possible_laws'].astype(str) == '[]']), " of ", len(docs_df),  "- unable to find plans.")

Only missing 8  of  824 - unable to find plans.


# Save

In [82]:
doi_dates_and_exemptions = docs_df.to_csv(os.path.join(data_path, 'doi_dates_and_exemptions_temp.csv'))